# TPV 13

* spontaneous rupture on a 60-degree dipping normal fault  
* homogeneous half-space: $V_p=5716\,m/s, V_s=3300\,m/s, \rho=2700\,kg/m^3$
* non-associative Drucker-Prager plasticity
* linear-slip weakening friction (LSW)
* initial stress conditions are depth-dependent
* prescribed nucleation zone: square, 1.5 km size

![](tpv13.jpg)

[Benchmark description (SCEC)](https://strike.scec.org/cvws/tpv12_13docs.html)

## Mesh generation

We use Gmsh for mesh generation. The following steps are required:

* Create a domain (box)
* Create a 60° dipping fault plane
* Create the nucleation patch
* Set boundary conditions

Gmsh comes with its own scripting language with which one can describe the geometry passed to the automatic mesh generator.
We provide the script tpv13_training.geo and are going to generate a mesh in the next step.

In [ ]:
!gmsh -3 tpv13_training.geo

You should now have a file called "tpv13_training.msh".
In the next step, we translate this file into the efficient HDF5 format, which is going to be read in by SeisSol.

In [18]:
!pumgen -s msh2 tpv13_training.msh

/bin/bash: pumgen: Kommando nicht gefunden.


The files "tpv13_training.puml.h5" and "tpv13_training.xdmf" were created.
While the .h5 file is read by SeisSol, the .xdmf file can be used to visualize the mesh, as in the following.

In [11]:
import vtk
import pyvista as pv
pv.set_jupyter_backend('panel')

/usr/lib/python3/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (4.0.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [12]:
reader = vtk.vtkXdmfReader()
reader.SetFileName('tpv13_training.xdmf')
reader.Update()
mesh = pv.wrap(reader.GetOutput())
pv.plot(mesh, cmap='BuGn', background='white', show_edges=True)

Note that the mesh is refined close to the fault.

### Exercises
* Inspect the tpv13_training.geo file and try to comprehend the steps. Consult the [Gmsh documentation](https://gmsh.info/doc/texinfo/gmsh.html) about the BooleanFragments operation.
* Create a finer resolved mesh by adjusting the h_fault parameter. Do not change the .geo file but use the [*-setnumber*](https://gmsh.info/doc/texinfo/gmsh.html#Command_002dline-options) argument of the *gmsh* command.

## Run the code

In the following we start SeisSol using the provided parameters file.
You should adjust the OMP_NUM_THREADS variable to match the number of cores on your PC.

In [43]:
!OMP_NUM_THREADS=4 SeisSol_Release_dhsw_4_elastic parameters.par

/bin/bash: SeisSol_Release_dhsw_4_elastic: Kommando nicht gefunden.


While SeisSol is running, you might want to checkout the [documentation of the input files](https://seissol.readthedocs.io/en/latest/parameter-file.html).

## Visualization

In [3]:
from ipywidgets import interact

In [13]:
@interact(t=(0.0, 8.0, 1.0))
def plot(t):
    reader = vtk.vtkXdmfReader()
    reader.SetFileName('output/tpv13-fault.xdmf')
    reader.UpdateTimeStep(t)
    mesh = pv.wrap(reader.GetOutput())
    plotter = pv.Plotter(notebook=True)
    plotter.set_background('white')
    plotter.add_mesh(mesh, cmap='BuGn', scalars='SRs')
    plotter.view_xz()
    plotter.show()

interactive(children=(FloatSlider(value=4.0, description='t', max=8.0, step=1.0), Output()), _dom_classes=('wi…

In [7]:
import pyvista as pv
pv.set_jupyter_backend('ipygany')

def create_mesh(value):
    res = int(value)
    sphere = pv.Sphere(phi_resolution=res, theta_resolution=res)
    p.add_mesh(sphere, name='sphere', show_edges=True)
    return

p = pv.Plotter(notebook=True)
p.add_slider_widget(create_mesh, [5, 100], title='Resolution')
p.show(jupyter_backend='ipygany')

Scene(background_color='#4c4c4c', camera={'position': [0.9656204413323679, 0.9656204413323679, 0.9656204413323…